<a href="https://colab.research.google.com/github/CarlosMendez1997Col/Automation_of_satellite_image_downloads_GeeMap_APIs_REST/blob/main/1.%20Sentinel/Automatizar_descarga_Sentinel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated script to download Sentinel-1 images from a Google Earth Engine using GeeMap



#Radiometric Calibration: sigma0
#Scale: decibel
#Pixel Spacing (spatial resolution): 10
#DEM Matching (geocoding): true
#Speckle filter: True


# Import libraries and packages

In [1]:
!pip install geemap
!pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.7 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import webbrowser
import ipywidgets as widgets
import geopandas as gpd
from datetime import datetime

## Autentication in Google Colab and GEE

In [3]:
auth_url = ee.Authenticate(auth_mode='notebook')
webbrowser.open(auth_url)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=16lFQxz5rzAooS_ONdslUQ7piX_7xIvsdfn6P7ZVlzw&tc=0z2wodtimg1d7bpASDwmEthZordwR21e6wq783s6Wfo&cc=QOGHE2yXqQ6lDPtTzwjm8zCYM8Pf7hytDDZh7UyIkPE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ASc3gC33xT_1Z_ZfZtilQhXDslbElfVwMScmg5X6ovnJVbToXPZjpDAXYrM

Successfully saved authorization token.


## Create basemaps and import Area Of Interest (AOI)

In [4]:
cobija_fc = ee.FeatureCollection("projects/gee-projects-481514/assets/CobijaSHP")

MapCobija = geemap.Map()
MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP")
MapCobija.centerObject(cobija_fc, 13)
MapCobija.setOptions('HYBRID')

styles = {
    "OpenStreetMap": "OpenStreetMap",
    "Esri World Street Map": "Esri.WorldStreetMap",
    "Esri World Imagery": "Esri.WorldImagery",
    "Esri World Topo Map": "Esri.WorldTopoMap",
    "Esri World Gray Canvas": "Esri.WorldGrayCanvas",
    "Esri World Shaded Relief": "Esri.WorldShadedRelief",
    "Esri World Terrain": "Esri.WorldTerrain",
    "CartoDB DarkMatter": "CartoDB.DarkMatter",
    "CartoDB Positron": "CartoDB.Positron",
    "CartoDB Voyager": "CartoDB.Voyager",
    "OpenTopoMap": "OpenTopoMap"
}

style_dropdown = widgets.Dropdown(
    options=list(styles.keys()),
    value="Esri World Imagery",
    description="Basemap:"
)

def update_style(change):
    MapCobija.add_basemap(styles[change['new']])
    MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP")
    MapCobija.centerObject(cobija_fc, 13)

style_dropdown.observe(update_style, names='value')

display(style_dropdown)
MapCobija

Dropdown(description='Basemap:', index=2, options=('OpenStreetMap', 'Esri World Street Map', 'Esri World Image…

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

# Search Sentinel Images in the AOI

## Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar Ground Range Detected

In [5]:
s1_collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(cobija_fc)

first_image = s1_collection.sort('system:time_start').first()
start_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
end_date = datetime.utcnow().strftime("%Y-%m-%d")

s1 = (s1_collection
      .filterDate(start_date, end_date)
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
      .filter(ee.Filter.eq('instrumentMode', 'IW'))
      .filter(ee.Filter.eq('productType', 'GRD'))
      .filter(ee.Filter.eq('platform_number', 'A'))
      .sort('system:time_start', False))

orbit_filter_pass = 'DESCENDING'
relative_orbit = 127

if orbit_filter_pass is not None:
    s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit_filter_pass))

if relative_orbit is not None:
    s1 = s1.filter(ee.Filter.eq('relativeOrbitNumber_start', relative_orbit))

## Print S1 images in AOI

In [6]:
images_list = s1.toList(s1.size())
info_list = []
for i in range(s1.size().getInfo()):
    img = ee.Image(images_list.get(i))
    info = img.getInfo()
    info_list.append({
        'ID': info['id'],
        'Fecha': info['properties']['system:time_start'],
        'Polarización': info['properties']['transmitterReceiverPolarisation']
    })

print("Imágenes Sentinel-1 disponibles en Cobija:")
for info in info_list:
    print(info)

Imágenes Sentinel-1 disponibles en Cobija:
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20260108T101431_20260108T101501_062674_07DBA5_DF0D', 'Fecha': 1767867271000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251227T101432_20251227T101502_062499_07D4E7_D566', 'Fecha': 1766830472000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251215T101433_20251215T101503_062324_07CE1A_87D7', 'Fecha': 1765793673000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251203T101435_20251203T101504_062149_07C742_86A4', 'Fecha': 1764756875000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251121T101436_20251121T101506_061974_07C06E_317E', 'Fecha': 1763720076000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251109T101436_20251109T101506_061799_07B9A0_2B70', 'Fecha': 1762683276000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251028T10143

## Visualize first 10 S1 images in AOI

In [7]:
s1_10 = s1.limit(10)
images_list_10 = s1_10.toList(s1_10.size())

MapCobijaS1 = geemap.Map()
for i in range(s1_10.size().getInfo()):
    img = ee.Image(images_list_10.get(i))
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    MapCobijaS1.addLayer(img.select('VV'), {"min": -20, "max": 0}, f"Sentinel-1 VV {date_str}")
    MapCobijaS1.addLayer(img.select('VH'), {"min": -20, "max": 0}, f"Sentinel-1 VH {date_str}")

MapCobijaS1.addLayer(cobija_fc, {"color":"darkblue", "opacity":0.7}, "Cobija SHP")
MapCobijaS1.centerObject(cobija_fc, 12)
MapCobijaS1

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Parámetros de exportación
scale = 10  # metros

In [ ]:


def export_image_local(image, pol, out_dir, date_str, orbit_char):
    """
    Exporta una banda (VV o VH) a un GeoTIFF local usando geemap.
    pol: 'VV' o 'VH'
    orbit_char: 'A' o 'D'
    """
    filename = os.path.join(out_dir, f"{date_str}_{pol}_{orbit_char}.tif")
    print(f"Descargando {filename} ...")

    geemap.ee_export_image(
        image,              # argumento posicional
        filename=filename,
        scale=scale,
        region=bbox,        # región en WGS84
        crs=utm_epsg,       # reproyección a UTM en la exportación
        file_per_band=False
    )

In [ ]:
# ============================================================================
# PROCESO PRINCIPAL
# ============================================================================

def main():
    col = prepare_collection()
    size = col.size().getInfo()
    print(f"Imágenes encontradas con los filtros dados: {size}")

    if size == 0:
        print("No se encontraron imágenes para los filtros especificados.")
        return

    # Para prueba rápida, descomenta:
    # size = min(size, 3)

    img_list = col.toList(size)

    for i in range(size):
        img = ee.Image(img_list.get(i))

        # Fecha para el nombre del archivo
        date = ee.Date(img.get('system:time_start'))
        date_str = date.format('yyyyMMdd').getInfo()

        # Tipo de órbita (ASCENDING/DESCENDING) y letra A/D
        orbit_pass = ee.String(img.get('orbitProperties_pass')).getInfo()
        orbit_char = 'A' if orbit_pass.upper().startswith('ASC') else 'D'

        # Bandas VV y VH (ya en dB en COPERNICUS/S1_GRD)
        vv = img.select('VV')
        vh = img.select('VH')

        # Exportar con nombres YYYYMMDD_VV/VH_A/D
        export_image_local(vv, "VV", output_dir_vv, date_str, orbit_char)
        export_image_local(vh, "VH", output_dir_vh, date_str, orbit_char)

    print("Proceso terminado.")

# Ejecutar
if __name__ == "__main__":
    main()

Imágenes encontradas con los filtros dados: 252
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170904_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170904_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170904_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170904_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170916_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170916_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170916_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170916_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170928_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170928_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170928_VH_D.tif ...
Generating URL

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180126_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180207_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180207_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180207_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180207_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180219_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180219_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180219_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180219_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180303_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180303_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180303_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180619_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180701_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180701_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180701_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180701_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180713_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180713_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180713_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180713_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180725_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180725_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180725_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181110_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181122_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181122_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181122_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181122_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181204_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181204_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181204_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181204_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181216_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181216_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181216_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190403_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190415_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190415_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190415_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190415_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190427_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190427_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190427_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190427_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190509_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190509_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190509_VH_D

...

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20231214_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20231226_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20231226_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20231226_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20231226_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240107_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240107_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240107_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240107_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240119_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240119_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240119_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240506_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240518_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240518_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240518_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240518_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240530_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240530_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240530_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240530_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240611_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240611_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240611_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240927_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241009_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241009_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241009_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241009_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241021_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241021_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241021_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241021_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241102_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241102_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241102_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250218_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250314_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250314_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250314_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250314_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250326_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250326_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250326_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250326_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250407_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250407_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250407_VH_D

EEException: Project ee-sebastianpalominoangel is not registered to use Earth Engine. Visit https://console.cloud.google.com/earth-engine/configuration?project=ee-sebastianpalominoangel to register your project. See https://developers.google.com/earth-engine/guides/access for more details. If you have previously registered your project for noncommercial use and lost access, you will need to re-verify your eligibility. See the https://developers.google.com/earth-engine/guides/access#configuring_noncommercial_access for more details.